In [1]:
# Install 0.4 preview from source
!pip install -q git+https://github.com/keras-team/keras-nlp.git tensorflow==2.11 --upgrade

     |████████████████████████████████| 588.3 MB 22 kB/s 
     |████████████████████████████████| 5.8 MB 76.6 MB/s 
     |████████████████████████████████| 439 kB 109.1 MB/s 
     |████████████████████████████████| 6.0 MB 103.0 MB/s 
     |████████████████████████████████| 1.7 MB 91.2 MB/s 


In [2]:
import keras_nlp
import tensorflow as tf
from tensorflow import keras

# Use mixed precision for optimal performance
keras.mixed_precision.set_global_policy('mixed_float16')

# KerasNLP: Modular NLP Workflows for Keras


KerasNLP is a natural language processing library that supports users through their entire development cycle. Our workflows are built from modular components that have SoTA preset weights and architectures when used out-of-the-box and are easily customizable when more control is needed.

This library is an extension of the core Keras API; all high level modules are [`Layers`](https://keras.io/api/layers/) or
[`Models`](https://keras.io/api/models/). If you are familiar with Keras, congratulations! You already understand most of KerasNLP.

This guide demonstrates our modular approach using a sentiment analysis example at six levels of complexity:

* Inference with a pretrained classifier
* Fine tuning a pretrained backbone
* Fine tuning with user-controlled preprocessing
* Fine tuning a custom model
* Pretraining a backbone model
* Build and train your own transformer from scratch

Throughout our guide we use Professor Keras, the official Keras mascot, as a visual reference for the complexity of the material:

![picture](https://drive.google.com/uc?id=1d14Qpmfgjf6zu4z30HBaonH8PYDHgVoU)



# API quickstart

Our highest level API is `keras_nlp.models`. These symbols cover the complete user journey of converting strings to tokens, tokens to dense features, and dense features to task-specific output. For each `XX` architecture (e.g., `Bert`), we offer the following modules:

* **Tokenizer**: `keras_nlp.models.XXTokenizer`
    * **What it does**: Converts strings to `tf.RaggedTensor`s of token ids.
    * **Why it's important**: The raw bytes of a string are too high dimensional to be useful features so we first map them to a small number of tokens, for example `"The quick brown fox"` to `["the", "qu", "##ick", "br", "##own", "fox"]`.
    * **Inherits from**: `keras.Layer`.
* **Preprocessor**: `keras_nlp.models.XXPreprocessor`
    * **What it does**: Converts strings to a dictonary of preprocessed tensors consumed by the backbone, starting with tokenization.
    * **Why it's important**: Each model uses special tokens and extra tensors to understand the input such as deliminting input segments and identifying padding tokens. Padding each sequence to the same length improves computational efficiency.
    * **Has a**: `XXTokenizer`.
    * **Inherits from**: `keras.Layer`.
* **Backbone**: `keras_nlp.models.XXBackbone`
    * **What it does**: Converts preprocessed tensors to dense features. *Does not handle strings; call the preprocessor first.*
    * **Why it's important**: The backbone distills the input tokens into dense features that can be used in downstream tasks. It is generally pretrained on a language modeling task using massive amounts of unlabeled data. Transfering this information to a new task is a major breakthrough in modern NLP.
    * **Inherits from**: `keras.Model`.
* **Task**: e.g., `keras_nlp.models.XXClassifier`
    * **What it does**: Converts strings to task-specific output (e.g., classification probabilities).
    * **Why it's important**: Task models combine string preprocessing and the backbone model with task-specific `Layers` to solve a problem such as sentence classification, token classification, or text generation. The additional `Layers` must be fine-tuned on labeled data.
    * **Has a**: `XXBackbone` and `XXPreprocessor`.
    * **Inherits from**: `keras.Model`.

Here is the modular hierarchy for `BertClassifier` (all relationships are compositional):

![picture](https://drive.google.com/uc?id=1vHBQ1oFbto8ItfhsLcxKhIwOIdJE1X9n)

All modules can be used independently and have a `from_preset()` method in addition to the standard constructor that instantiates the class with **preset** architecture and weights (see examples below).

# Data

We will use a running example of sentiment analysis of IMDB movie reviews. In this task, we use the text to predict whether the review was positive (`label = 1`) or negative (`label = 0`).

We load the data using `keras.utils.text_dataset_from_directory`, which utilizes the powerful `tf.data.Dataset` format for examples.

In [3]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
# Remove unsupervised examples
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  31.1M      0  0:00:02  0:00:02 --:--:-- 31.1M


In [4]:
BATCH_SIZE = 16
imdb_train = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
)
imdb_test = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=BATCH_SIZE,
)

# Inspect first review
# Format is (review text tensor, label tensor)
print(imdb_train.unbatch().take(1).get_single_element())

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
(<tf.Tensor: shape=(), dtype=string, numpy=b'I avoided watching this film for the longest time. Long before it was even released I had dismissed it as an over-hyped, over-blown, overly romanticized piece of Hollywood schmaltz, and I wanted nothing to do with it. I never watched it in the theatre. I shook my head in disbelief at the 11 Academy Awards - even though I had never seen it. Then I was asked to be a judge at a high school public speaking contest. One of the girls spoke about this movie. "It was so great," she said. "You really felt like you were on the ship." "Nonsense," I thought. I shared my feelings with my fellow judges. One looked at me and said, "you might be right, but if she liked the movie that much maybe she\'ll want to learn more about the real Titanic. The movie must have done something right to get her so interested." "Well, maybe," thought I. Then it finally appeared on Pay TV. "O

# Inference with a pretrained classifier

![picture](https://drive.google.com/uc?id=1xeMHVCxYhm3_oC37Gg7k0bG-yhsVr0Dv)

The highest level module in KerasNLP is a **task**. A **task** is a `keras.Model` consisting of a (generally pretrained) **backbone** model and task-specific layers. Here's an example using `keras_nlp.models.BertClassifier`.

**Note**: Outputs are the logits per class (e.g., `[0, 0]` is 50% chance of positive).



In [ ]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased_sst2")
# Note: batched inputs expected so must wrap string in iterable
classifier.predict(["I love modular workflows in keras-nlp!"])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


1/1 [==============================] - 2s 2s/step


array([[-1.54 ,  1.543]], dtype=float16)

All **tasks** have a `from_preset` method that constructs a `keras.Model` instance with preset preprocessing, architecture and weights. This means that we can pass raw strings in any format accepted by a `keras.Model` and get output specific to our task.

This particular **preset** is a `"bert_tiny_uncased_en"` **backbone** fine-tuned on `sst2`, another movie review sentiment analysis (this time from Rotten Tomatoes). We use the `tiny` architecture for demo purposes, but larger models are recommended for SoTA performance. For all the task-specific presets available for `BertClassifier`, see [keras.io](https://resilient-dango-43f7b8.netlify.app/api/keras_nlp/models/).

Let's evaluate our classifier on the IMDB dataset. We first need to compile the `keras.Model`. The output is `[loss, accuracy]`,

**Note**: We don't need an optimizer since we're not training the model.

In [ ]:
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=keras.metrics.SparseCategoricalAccuracy(),
    jit_compile=True,
)

classifier.evaluate(imdb_test)

1563/1563 [==============================] - 50s 30ms/step - loss: 0.4630 - sparse_categorical_accuracy: 0.7836


[0.4629538357257843, 0.7836400270462036]

Our result is 78% accuracy without training anything. Not bad!

# Fine tuning a pretrained BERT backbone

![picture](https://drive.google.com/uc?id=1YytOYRSqsrhJ4NLatVOSuVMbLPa9iXrw)

When labeled text specific to our task is available, fine-tuning a custom classifier can improve performance. If we want to predict IMDB review sentiment, using IMDB data should perform better than Rotten Tomatoes data! And for many tasks no relevant pretrained model will be available (e.g., categorizing customer reviews).

The workflow for fine-tuning is almost identical to above, except that we request a **preset** for the **backbone**-only model rather than the entire classifier. When passed a **backone** **preset**, a **task** `Model` will randomly initialize all task-specific layers in preparation for training. For all the **backbone** presets available for `BertClassifier`, see [keras.io](https://resilient-dango-43f7b8.netlify.app/api/keras_nlp/models/).

To train your classifier, use `Model.compile()` and `Model.fit()` as with any other `keras.Model`. Since preprocessing is included in all **tasks** by default, we again pass the raw data.


In [ ]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
)
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.experimental.AdamW(5e-5),
    metrics=keras.metrics.SparseCategoricalAccuracy(),
    jit_compile=True,
)
classifier.fit(
    imdb_train,
    validation_data=imdb_test,
    epochs=1,
)

1563/1563 [==============================] - 301s 182ms/step - loss: 0.4116 - sparse_categorical_accuracy: 0.8052 - val_loss: 0.3101 - val_sparse_categorical_accuracy: 0.8685


Here we see significant lift in validation accuracy (0.78 -> 0.87) with a single epoch of training even though the IMDB dataset is much smaller than `sst2`.


# Fine tuning with user-controlled preprocessing
![picture](https://drive.google.com/uc?id=1T_40vtl8daihS-kKYTFWejFd19KJAyDK)

For some advanced training scenarios, users might prefer direct control over preprocessing. For large datasets, examples can be preprocessed in advance and saved to disk or preprocessed by a separate worker pool using `tf.data.experimental.service`. In other cases, custom preprocessing is needed to handle the inputs.

Pass `preprocessor=None` to the constructor of a **task** `Model` to skip automatic preprocessing or pass a custom `BertPreprocessor` instead.



## Separate preprocessing from the same preset

Each model architecture has a parallel **preprocessor** `Layer` with its own `from_preset` constructor. Using the same **preset** for this `Layer` will return the matching **preprocessor** as the **task**.

In this workflow we train the model over three epochs using `tf.data.Dataset.cache()`, which computes the preprocessing once and caches the result before fitting begins.

**Note:** this code only works if your data fits in memory. If not, pass a `filename` to `cache()`.

In [9]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=512,
)

imdb_train_cached = imdb_train.map(
    preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
imdb_test_cached = imdb_test.map(
    preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)

classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    preprocessor=None,
)
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.experimental.AdamW(5e-5),
    metrics=keras.metrics.SparseCategoricalAccuracy(),
    jit_compile=True,
)
classifier.fit(
    imdb_train_cached,
    validation_data=imdb_test_cached,
    epochs=3,
)

17602216/17602216 [==============================] - 0s 0us/step
Epoch 1/3
1563/1563 [==============================] - 277s 166ms/step - loss: 0.4129 - sparse_categorical_accuracy: 0.8084 - val_loss: 0.3361 - val_sparse_categorical_accuracy: 0.8585
Epoch 2/3
1563/1563 [==============================] - 238s 152ms/step - loss: 0.2640 - sparse_categorical_accuracy: 0.8941 - val_loss: 0.3277 - val_sparse_categorical_accuracy: 0.8689
Epoch 3/3
1563/1563 [==============================] - 237s 152ms/step - loss: 0.1954 - sparse_categorical_accuracy: 0.9282 - val_loss: 0.3837 - val_sparse_categorical_accuracy: 0.8631


After three epochs, our validation accuracy has only increased to 0.88. This is both a function of the small size of our dataset and our model. To exceed 90% accuracy, try larger **presets** such as  `"bert_base_en_uncased"`.

## Custom preprocessing

In cases where custom preprocessing is required, we offer direct access to the `Tokenizer` class that maps raw strings to tokens. It also has a `from_preset` constructor to get the vocabulary matching pretraining.

**Note:** `BertTokenizer` does not pad sequences by default, so the output is `tf.RaggedTensor`.



In [6]:
tokenizer = keras_nlp.models.BertTokenizer.from_preset("bert_tiny_en_uncased")
tokenizer(["I love modular workflows!", "Libraries over frameworks!"])

<tf.RaggedTensor [[1045, 2293, 19160, 2147, 12314, 2015, 999],
 [8860, 2058, 7705, 2015, 999]]>

In [7]:
# Write your own packer or use one our `Layers`
packer = keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    # Note: This cannot be longer than the preset's `sequence_length`, and there
    # is no check for a custom preprocessor!
    sequence_length=64,
)

def preprocessor(x, y):
    token_ids, segment_ids = packer(tokenizer(x))
    x = {
        "token_ids": token_ids,
        "segment_ids": segment_ids,
        "padding_mask": token_ids != 0,
    }
    return x, y

imbd_train_preprocessed = imdb_train.map(
    preprocessor, tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
imdb_test_preprocessed = imdb_test.map(
    preprocessor, tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

# Preprocessed example
print(imbd_train_preprocessed.unbatch().take(1).get_single_element())

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


({'token_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([  101,  2130,  2065,  2009,  2020, 19512,  6057,  1010,  2023,
        9587, 21285,  2100, 13844,  6198,  1997,  1037,  2143,  2052,
        2145,  2022, 17358, 15787,  4487, 21338,  2229,  5051,  6593,
        3993,  1012,  7458,  2074,  2460,  1997, 10443,  2039,  1996,
        3337,  1005, 18113,  1998,  2128,  1011,  4372, 18908,  2075,
        1005,  5353,  2012, 15941,  1005,  1055,  1005,  2021,  2069,
        2074,  3135,  6554, 25546,  1998,  1996,  2472,  1997,  1996,
         102], dtype=int32)>, 'segment_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)>, 'padding_mask': <tf.Tensor: shape=(64,), dtype=bool, numpy=
array([ True,  True,  True,  True,  True,  True,  True,  True,  T

# Fine tuning with a custom model
![picture](https://drive.google.com/uc?id=1T_40vtl8daihS-kKYTFWejFd19KJAyDK)

For more advanced applications, an appropriate **task** `Model` may not be available. In this case we provide direct access to the **backbone** `Model`, which has its own `from_preset` constructor and can be composed with custom `Layer`s. Detailed examples can be found at https://keras.io/guides/transfer_learning/.

A **backbone** `Model` does not include automatic preprocessing but can be paired with a matching **preprocessor** using the same **preset** as shown in the previous workflow.

In this workflow we experiment with freezing our backbone model and adding two trainable transfomer layers to adapt to the new input.

**Note**: We can igonore the warning about gradients for the `pooled_dense` layer because we are using BERT's sequence output.


In [ ]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_tiny_en_uncased")
backbone = keras_nlp.models.BertBackbone.from_preset("bert_tiny_en_uncased")

imdb_train_preprocessed = imdb_train.map(
    preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
imdb_test_preprocessed = imdb_test.map(
    preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)

backbone.trainable = False
inputs = backbone.input
sequence = backbone(inputs)["sequence_output"]
for _ in range(2):
  sequence = keras_nlp.layers.TransformerEncoder(
      num_heads=2,
      intermediate_dim=512,
      dropout=0.1,
  )(sequence)
# Use [CLS] token output to classify
outputs = keras.layers.Dense(2)(sequence[:, backbone.cls_token_index, :])

model = keras.Model(inputs, outputs)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.experimental.AdamW(5e-5),
    metrics=keras.metrics.SparseCategoricalAccuracy(),
    jit_compile=True,
)
model.summary()
model.fit(
    imdb_train_preprocessed,
    validation_data=imdb_test_preprocessed,
    epochs=3,
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 padding_mask (InputLayer)      [(None, None)]       0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, None)]       0           []                               
                                                                                                  
 token_ids (InputLayer)         [(None, None)]       0           []                               
                                                                                                  
 bert_backbone_3 (BertBackbone)  {'sequence_output':  4385920    ['padding_mask[0][0]',           
                                 (None, None, 128),               'segment_ids[0][0]',        

This model achieves reasonable accuracy despite having only 10% the trainable parameters of our `BertClassifier` model. Each training step takes about 1/3 of the time---even accounting for cached preprocessing.

# Pretraining a backbone model
![picture](https://drive.google.com/uc?id=1pzwLPCtvzmHY3DKzH-MBzmjWFJ3pKVB5)

Do you have access to large unlabeled datasets in your domain? Are they are around the same size as used to train popular backbones such as BERT, RoBERTa, or GPT2 (XX+ GiB)? If so, you might benefit from domain-specific pretraining of your own backbone models.

NLP models are generally pretrained on a language modeling task, predicting masked words given the visible words in an input sentence. For example, given the input `"The fox [MASK] over the [MASK] dog"`, the model might be asked to predict `["jumped", "lazy"]`. The lower layers of this model are then packaged as a **backbone** to be combined with layers relating to a new task.

The KerasNLP library offers SoTA **backbones** and **tokenizers** to be trained from scratch without presets.

In this workflow we pretrain a BERT **backbone** using our IMDB review text. We skip the "next sentence prediction" (NSP) loss because it adds significant complexity to the data processing and was dropped by later models like RoBERTa. See our e2e [BERT pretraining example](https://github.com/keras-team/keras-nlp/tree/4f9ebefa82af22b4f4267dfa80fa525f7a03bd5d/examples/bert) for step-by-step details on how to replicate the original paper.

## Preprocessing

In [15]:
# All BERT `en` models have the same vocabulary, so reuse preprocessor from
# "bert_tiny_en_uncased"
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=256,
)
packer = preprocessor.packer
tokenizer = preprocessor.tokenizer

# keras.Layer to replace some input tokens with the "[MASK]" token
masker = keras_nlp.layers.MLMMaskGenerator(
    vocabulary_size=tokenizer.vocabulary_size(),
    mask_selection_rate=0.25,
    mask_selection_length=64,
    mask_token_id=tokenizer.token_to_id("[MASK]"),
    unselectable_token_ids=[
        tokenizer.token_to_id(x) for x in ["[CLS]", "[PAD]", "[SEP]"]
    ],
)

def preprocess(inputs, label):
    inputs = preprocessor(inputs)
    masked_inputs = masker(inputs["token_ids"])
    # Split the masking layer outputs into a (features, labels, and weights)
    # tuple that we can use with keras.Model.fit().
    features = {
        "token_ids": masked_inputs["token_ids"],
        "segment_ids": inputs["segment_ids"],
        "padding_mask": inputs["padding_mask"],
        "mask_positions": masked_inputs["mask_positions"],
    }
    labels = masked_inputs["mask_ids"]
    weights = masked_inputs["mask_weights"]
    return features, labels, weights

pretrain_ds = imdb_train.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)
pretrain_val_ds = imdb_test.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

# Tokens with ID 103 are "masked"
print(pretrain_ds.unbatch().take(1).get_single_element())

({'token_ids': <tf.Tensor: shape=(256,), dtype=int32, numpy=
array([  101,  2702,  2041,  1997,   103,  2340,  2460,  3152,  1999,
        2023,  3185,  2024, 17743,  8815,   103,  1045,   103,  2069,
         103,   103,  2028, 15640,   103,  1012,  2358, 24449,  2135,
        1010,  2035,   103,  1996,   103,  2472,  4900,  2000, 17279,
        1996,   103,  1997,  3633,  2030,  2967,  1999,  4434,  2007,
        1023,  1011,  2340,  1024,  1996,   103,  8711,  1010, 12419,
         103,  1010, 21524,   103,  1996, 24835,  1997,  5034, 15878,
        7389,  5555,  1010,   103,   103,  5635,  1998,  7897,  1999,
        3088,  1010,   103, 23555,  3215,  8648,  1998, 13831,   103,
         103,  2232,  1010,   103,   103,  1999,  3956,  1010, 27890,
        1011,  2718,  1998,  2110,  1011, 27666,  5152,  4841,  1999,
        1996,  3915,  1010,  2214,  2111,  2542,  2894,   103,   103,
         103, 10530,   103, 25755,  1999,  1996,  8072,  1997,   103,
        3548,  1012,  2023,  

## Pretraining model

In [16]:
# BERT backbone
backbone = keras_nlp.models.BertBackbone(
    vocabulary_size=tokenizer.vocabulary_size(),
    num_layers=2,
    num_heads=2,
    hidden_dim=128,
    intermediate_dim=512,
)

# Language modeling head
mlm_head = keras_nlp.layers.MLMHead(
    embedding_weights=backbone.token_embedding.embeddings,
)

inputs = {
    "token_ids": keras.Input(shape=(None,), dtype=tf.int32),
    "segment_ids": keras.Input(shape=(None,), dtype=tf.int32),
    "padding_mask": keras.Input(shape=(None,), dtype=tf.int32),
    "mask_positions": keras.Input(shape=(None,), dtype=tf.int32),
}

# Encoded token sequence
sequence = backbone(inputs)["sequence_output"]

# Predict an output word for each masked input token.
# We use the input token embedding to project from our encoded vectors to
# vocabulary logits, which has been shown to improve training efficiency.
outputs = mlm_head(sequence, mask_positions=inputs["mask_positions"])

# Define and compile our pretraining model.
pretraining_model = keras.Model(inputs, outputs)
pretraining_model.summary()
pretraining_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.experimental.AdamW(learning_rate=5e-4),
    weighted_metrics=keras.metrics.SparseCategoricalAccuracy(),
    jit_compile=True,
)

# Pretrain on IMDB dataset
pretraining_model.fit(
    pretrain_ds,
    validation_data=pretrain_val_ds,
    epochs=3,    # Increase to 6 for higher accuracy
)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_11 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_10 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, None)]       0           []                               
                                                                                            

1563/1563 [==============================] - 119s 67ms/step - loss: 5.2670 - sparse_categorical_accuracy: 0.0846 - val_loss: 4.9760 - val_sparse_categorical_accuracy: 0.1162
Epoch 2/3
1563/1563 [==============================] - 93s 60ms/step - loss: 4.9556 - sparse_categorical_accuracy: 0.1244 - val_loss: 4.8710 - val_sparse_categorical_accuracy: 0.1301
Epoch 3/3
1563/1563 [==============================] - 93s 60ms/step - loss: 4.8342 - sparse_categorical_accuracy: 0.1406 - val_loss: 4.6404 - val_sparse_categorical_accuracy: 0.1709


After pretraining save your `backbone` submodel to use in a new task!

# Build and train your own transformer from scratch
![picture](https://drive.google.com/uc?id=1pzwLPCtvzmHY3DKzH-MBzmjWFJ3pKVB5)

Want to implement a novel transformer architecture? The KerasNLP library offers all the low-level modules used to build SoTA architectures in our `models` API. This includes the `keras_nlp.tokenizers` API which allows you to train your own subword tokenizer using `WordPieceTokenizer`, `BytePairTokenizer`, or `SentencePieceTokenizer`.

In this workflow we train a custom tokenizer on the IMDB data and design a backbone with custom transformer architecture. For simplicity we then train directly on the classification task. Interested in more details? We wrote an entire guide to pretraining and finetuning a custom transformer: https://keras.io/guides/keras_nlp/transformer_pretraining/

## Train custom vocabulary from IMBD data

In [ ]:
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    imdb_train.map(lambda x, y: x),
    vocabulary_size=20_000,
    lowercase=True,
    strip_accents=True,
    reserved_tokens=["[PAD]", "[START]", "[END]", "[MASK]", "[UNK]"],
)
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=True,
    strip_accents=True,
    oov_token="[UNK]",
)


## Preprocess data with custom tokenizer

In [ ]:
packer = keras_nlp.layers.StartEndPacker(
    start_value=tokenizer.token_to_id("[START]"),
    end_value=tokenizer.token_to_id("[END]"),
    pad_value=tokenizer.token_to_id("[PAD]"),
    sequence_length=512,
)

def preprocess(x, y):
    token_ids = packer(tokenizer(x))
    return token_ids, y

imdb_preproc_train_ds = imdb_train.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)
imdb_preproc_val_ds = imdb_test.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

print(imdb_preproc_train_ds.unbatch().take(1).get_single_element())

(<tf.Tensor: shape=(512,), dtype=int32, numpy=
array([    1,    96,   245,  4520,  9667,   100,   144,    98,    43,
        3590,    99,    96,  2201,    17,   392, 17025,    98,    96,
          11,  5362,   165,   192, 11408,  6869,  7655,    98,    96,
         535,  1636,    12,  1604,   986, 18649,  6623,  9654,    13,
          18,  1486,   127,    51,   134,  1146,    16,   131,    11,
          61,   174,  7595,   133,  3241,  2217,    98,  7668,   775,
        6263,  4281,    16,  2056,   136,  1625,   373,    16,   978,
         136, 11070,  1720,  3267,    16,  1048,     6,  3256, 15500,
       14959,     6,  4942,    16,    97,  2443,  6453,    18,   104,
         737, 12968,  1314,  1169,   129,  4226, 12968,    97,    60,
        7150, 15515,   100,   237,   100,   147,   544, 16477,   109,
         499,  8007,   122,   281,    18,   124,   122,   578,   128,
         166,   130,    96,   238,   244,   105,   416,    98,   163,
        1405,   124,   189,   961,   120,  


## Design a tiny transformer

In [ ]:
token_id_input = keras.Input(
    shape=(None,), dtype="int32", name="token_ids",
)
outputs = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=len(vocab),
    sequence_length=packer.sequence_length,
    embedding_dim=64,
)(token_id_input)
outputs = keras_nlp.layers.TransformerEncoder(
    num_heads=2,
    intermediate_dim=128,
    dropout=0.1,
)(outputs)
# Use "[START]" token to classify
outputs = keras.layers.Dense(2)(outputs[:, 0, :])
model = keras.Model(
    inputs=token_id_input,
    outputs=outputs,
)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 token_ids (InputLayer)      [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 64)         1259648   
 g_1 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_encoder_3 (Tran  (None, None, 64)         33472     
 sformerEncoder)                                                 
                                                                 
 tf.__operators__.getitem_7   (None, 64)               0         
 (SlicingOpLambda)                                               
                                                                 
 dense_2 (Dense)             (None, 2)                 130 

## Train the transformer directly on the classification objective

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.experimental.AdamW(5e-5),
    metrics=keras.metrics.SparseCategoricalAccuracy(),
    jit_compile=True,
)
model.fit(
    imdb_preproc_train_ds,
    validation_data=imdb_preproc_val_ds,
    epochs=3,
)

Epoch 1/3
1563/1563 [==============================] - 129s 78ms/step - loss: 0.6229 - sparse_categorical_accuracy: 0.6260 - val_loss: 0.4061 - val_sparse_categorical_accuracy: 0.8246
Epoch 2/3
1563/1563 [==============================] - 120s 76ms/step - loss: 0.3126 - sparse_categorical_accuracy: 0.8726 - val_loss: 0.3339 - val_sparse_categorical_accuracy: 0.8629
Epoch 3/3
1563/1563 [==============================] - 119s 76ms/step - loss: 0.2446 - sparse_categorical_accuracy: 0.9053 - val_loss: 0.3147 - val_sparse_categorical_accuracy: 0.8730


Excitingly, our custom classifier is similar to the performance of fine-tuning `"bert_tiny_en_uncased"`! To see the advantages of pretraining and exceed 90% accuracy we would need to use larger **presets** such as `"bert_base_en_uncased"`.